<a href="https://colab.research.google.com/github/DL-WG/ROMS-tutorial/blob/main/PC_AE_FPTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Flow past the cylinder with PC-based Autoencoder
## PREMIERE tutorial on reduced-order models
## Author: César Quilodrán-Casas

import numpy as np
import tensorflow.keras as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras import backend
import eofs
from eofs.standard import Eof

plt.rcParams.update({'font.size': 10})
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('axes', labelsize=10)

#Load vtk packages
import vtk
import sys, os
sys.path.append('/Users/cequilod/')
import vtktools

directory = '/Users/cequilod/ROMS-tutorial/'

#Load data
vel_x = np.load(directory + 'vx_field.npy')
vel_y = np.load(directory + 'vy_field.npy')

modelData = np.concatenate([vel_x, vel_y], 1)

#Add PCA
#Standardise data
meanData = np.mean(modelData, axis = 0)
stdData = np.std(modelData)
modelDataScaled = (modelData - meanData)/stdData

solver = Eof(modelDataScaled)

varianceCumulative = np.cumsum(solver.varianceFraction())
eigenvalues = solver.eigenvalues()
pcs = solver.pcs()
eof = solver.eofs()

#Scale data
modelMin = np.min(pcs, 0)
modelMax = np.max(pcs, 0)

def scaler(x, xmin, xmax, min, max):
    scale = (max - min)/(xmax - xmin)
    xScaled = scale*x + min - xmin*scale
    return xScaled

pcsScaled = scaler(pcs, modelMin, modelMax, 0, 1)

nFeatures = pcsScaled.shape[1]
nSnapshots = pcsScaled.shape[0]

#Get training and test data

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pcsScaled, pcsScaled, test_size=0.1, shuffle=True, random_state=42)

#Define model
input_enc = tf.Input(shape=(nFeatures))
enc = tf.layers.Dense(64)(input_enc)
enc = tf.layers.Dense(32)(enc)
enc_output = tf.layers.Dense(16)(enc)

input_dec = tf.Input(shape=(16))
dec = tf.layers.Dense(32)(input_dec)
dec = tf.layers.Dense(64)(dec)
dec = tf.layers.Dense(nFeatures, activation='sigmoid')(dec)

#Encoder model
enc_model = tf.Model(input_enc, enc_output)
enc_model.summary()
#Decoder model
dec_model = tf.Model(input_dec, dec)
dec_model.summary()
#ae model
ae_model = tf.Model(input_enc, dec_model(enc_output))

ae_model.compile(loss='mse', optimizer='adam')
history = ae_model.fit(X_train, y_train, epochs=100, batch_size=256, verbose=2, validation_data = (X_test, y_test), shuffle = True)
plt.plot(ae_model.predict(pcsScaled)[:, 0])
plt.plot(pcsScaled[:, 0])

#tf.models.save_model(ae_model, directory + 'ae_model')
#tf.models.save_model(enc_model, directory + 'enc_model')
#tf.models.save_model(dec_model, directory + 'dec_model')

ae_model = tf.models.load_model(directory + 'ae_model')
enc_model = tf.models.load_model(directory + 'enc_model')
dec_model = tf.models.load_model(directory + 'dec_model')

#Plot losses
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

#Prediction with AE
ae_prediction = ae_model.predict(pcsScaled)
def inverseScaler(xscaled, xmin, xmax, min, max):
    scale = (max - min) / (xmax - xmin)
    xInv = (xscaled/scale) - (min/scale) + xmin
    return xInv

ae_prediction = inverseScaler(ae_prediction, modelMin, modelMax, 0, 1)
ae_prediction = np.matmul(ae_prediction, eof)*stdData + meanData

#Predicted velocities
vel_x_pred = ae_prediction[:, :vel_x.shape[1]]
vel_y_pred = ae_prediction[:, vel_x.shape[1]:vel_y.shape[1]*2]

#Vector expects a third field, since this is 2D then vel_z = 0
vel_z = np.zeros((vel_x_pred.shape[0], vel_x.shape[1]))

vel_pred = np.zeros((vel_x_pred.shape[0], vel_x_pred.shape[1], 3))
vel_pred[:, :, 0] = vel_x_pred
vel_pred[:, :, 1] = vel_y_pred
vel_pred[:, :, 2] = vel_z

#Reconstruction onto .vtu files
for i in range(vel_pred.shape[0]):
    filename = directory + str(i) + '.vtu'
    ug = vtktools.vtu(filename)
    ug.AddVectorField('Recon_PCAE', vel_pred[i, :, :])
    # ug.AddScalarField('WGANAE_Tracer_8', np.squeeze(ae_prediction))
    # ug.AddScalarField('PC_Tracer_8', np.squeeze(pc_trun))
    ug.Write(directory + str(i) + '.vtu')
    print(i)

#Paraview